In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from projet_sdp.model import Model
from projet_sdp.loader import Data, load_data_from_json
from projet_sdp.generator import generate_random
from projet_sdp.ploter import Ploter

In [ ]:
# data: Data = load_data_from_json("../data/toy_instance.json")
data: Data = generate_random(5, 3, 3, 3)

In [ ]:
print(data)

In [ ]:
m = Model(data)

In [ ]:
m.constraint_cant_work_more_than_needed()
m.constraint_cant_work_on_days_off()
m.constraint_no_qual_work()
m.constraint_one_project_per_day()

In [ ]:
m.determine_affecte()
m.determine_debute()
m.determine_realise()

In [ ]:
m.objective_max_gain()

In [ ]:
m.update()

In [ ]:
m.optimize()

In [ ]:
def get_completion_day(p):
    for j in range(m.data.N):
        if m.realise(p, j).X == 1:
            return j - 1
    return None


def get_work(p):
    df = pd.DataFrame()
    for e in range(m.data.E):
        work_done = []
        for j in range(m.data.N):
            work = []
            for q in range(m.data.Q):
                work.append(m.projet(e, j, p, q).X)
            has_worked = 1 in work
            if not has_worked:
                if j in m.data.conges(e):
                    work_done.append("X")
                else:
                    work_done.append(" ")
            else:
                work = np.argmax(work)
                work = m.data.qualification_name(work)
                work_done.append(work)
        df[m.data.employee_name(e)] = work_done

    due = []
    for j in range(m.data.N):
        if j == m.data.due_date(p):
            due.append("X")
        else:
            due.append("")

    completed = []
    completion_day = get_completion_day(p)
    for j in range(m.data.N):
        if j == completion_day:
            completed.append("X")
        else:
            completed.append("")

    df["Due_Date"] = due
    df["Completed"] = completed

    return df

In [ ]:
for p in range(m.data.P):
    qualis = []
    for q in range(m.data.Q):
        qualis.append((m.data.qualification_name(q), m.data.duree(p, q)))
    print("Work on", m.data.projet_name(p), qualis)
    print(get_work(p))
    completion_day = get_completion_day(p)
    if completion_day != None:
        print("Project completed, gained", m.data.gain(p))
        if m.data.due_date(p) < completion_day:
            print("Project late", completion_day - m.data.due_date(p), "day, lost", m.data.penalite(p, completion_day))
        print("Total gain", m.data.gain(p) - m.data.penalite(p, completion_day))
    else:
        print("Project not completed")
        print("Project late", m.data.N - 1 - m.data.due_date(p), "day, lost", m.data.penalite(p, m.data.N - 1))
        print("Total gain", -m.data.penalite(p, m.data.N - 1))

    print()

print(m.model.ObjVal)

In [ ]:
# P: number of projects      -> p
# E: number of employees     -> e
# N: number of days          -> j
# Q: number of qualification -> q
print(m.data.N)

In [ ]:
plot: Ploter = Ploter(m)

plot.gantt_projects()

In [ ]:
plot.gantt_employees()